<a href="https://colab.research.google.com/github/Rami-RK/Retrieval_Augmented_Generation_RAG/blob/main/RAG_Step_3_Retrieval_QandA_Integrated_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **RAG: Step 3 - Retrieval Q&A Integrated with LLM**

### **Objectives**
At the end of the experiment you will be able to :


1. integrate retrieval Q&A from Vector DB with LLM
2. adress issue with simple similarity search by impelenting MMR technique
3. add memory to RAG
4. implement full fledge RAG and chat with the document

### **Installing and importing packages**

In [ ]:
!pip install openai

In [ ]:
!pip install langchain

In [ ]:
!pip install pypdf

In [ ]:
!pip install chromadb

In [ ]:
!pip install tiktoken

In [ ]:
import os
import openai
import numpy as np

#### **Authentication for OpenAI API**

In [ ]:
f = open('/content/openapi_key.txt')
api_key = f.read()
type(api_key)

str

In [ ]:
os.environ['OPENAI_API_KEY'] = api_key
openai.api_key= os.getenv('OPENAI_API_KEY')

### **Upload the vector db from previous step and unzip**

In [ ]:
!unzip /content/docs.zip  -d /

### **Similarity Search**

In [43]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/' #/content/content/docs //content/docs

In [118]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [101]:
print(vectordb._collection.count())

10


In [116]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [119]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [120]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [121]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

### **Addressing Diversity - MMR-Maximum Marginal Relevance**

Last class we introduced one problem: how to enforce diversity in the search results.

`Maximum marginal relevance` strives to achieve both relevance to the query *and diversity* among the results.

In [122]:
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)
## Expecting this as one result : ""A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.')]

### **Using the Vector DB created using doc1 and doc2**

In [123]:
question = "What does it say about Mahatma Gandhi?"
docs_ss = vectordb.similarity_search(question,k=3)

In [124]:
docs_ss[0].page_content[:500]

"India's struggle for independence from British colonial rule is a significant chapter in its history. \nLed by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement \ngained momentum and eventually led to India's independence on August 15, 1947. This day is \ncelebrated annually as Independence Day.\nIndia's economy is one of the fastest-growing in the world. It has transitioned from an agrarian \neconomy to a service-oriented and industrialized economy. The country is known "

In [125]:
docs_ss[1].page_content[:500]

"India's struggle for independence from British colonial rule is a significant chapter in its history. \nLed by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement \ngained momentum and eventually led to India's independence on August 15, 1947. This day is \ncelebrated annually as Independence Day.\nIndia's economy is one of the fastest-growing in the world. It has transitioned from an agrarian \neconomy to a service-oriented and industrialized economy. The country is known "

Note the difference in results with `MMR`.

In [126]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=2)

In [127]:
docs_mmr[0].page_content[:500]

"India's struggle for independence from British colonial rule is a significant chapter in its history. \nLed by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement \ngained momentum and eventually led to India's independence on August 15, 1947. This day is \ncelebrated annually as Independence Day.\nIndia's economy is one of the fastest-growing in the world. It has transitioned from an agrarian \neconomy to a service-oriented and industrialized economy. The country is known "

In [128]:
docs_mmr[1].page_content[:500]

"India's cultural heritage is preserved in its ancient monuments and UNESCO World Heritage \nSites. From the intricate carvings of Khajuraho temples to the majestic forts of Rajasthan, these\narchitectural marvels reflect India's rich history and artistic traditions.\nIndia's diversity extends to its languages as well. While Hindi and English are the official \nlanguages at the national level, there are 22 officially recognized regional languages, including \nBengali, Tamil, Telugu, Marathi, Urdu, Pun"

### **Addressing Specificity: Working with metadata - Manually**

In last notebook, we showed that a question about the 2nd doc can include results from other lectures as well.

To address this, many vectorstores support operations on `metadata`.

`metadata` provides context for each embedded chunk.

In [46]:
question = "what did they say about about Tourism?"

In [129]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":'/content/doc2.pdf'} # manually passing metadata, using metadata filter.
)

In [130]:
docs

[Document(page_content="India's diplomatic influence is also growing on the global stage. The country actively \nparticipates in international forums and has strong bilateral relations with nations around the \nworld. India is a founding member of the Non-Aligned Movement and plays an active role in \nvarious international organizations, such as the United Nations and World Trade Organization.\nIn conclusion, India is a vast and diverse country with a rich cultural heritage, stunning \nlandscapes, and a rapidly growing economy. It is a nation where ancient traditions coexist with \nmodern aspirations. Despite its challenges, India continues to evolve and leave an indelible \nmark on the world, making it a fascinating and dynamic country to explore.", metadata={'page': 1, 'source': '/content/doc2.pdf'}),
 Document(page_content='However, India also faces various socio-economic challenges. Poverty, income inequality, and \nunemployment are persistent issues that the country strives to add

In [131]:
for d in docs:
    print(d.metadata)

{'page': 1, 'source': '/content/doc2.pdf'}
{'page': 0, 'source': '/content/doc2.pdf'}
{'page': 0, 'source': '/content/doc2.pdf'}


### **Addressing Specificity: Working with metadata using self-query retriever -Automatically**

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:

1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [51]:
!pip install lark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.1 MB/s eta 0:00:00


In [52]:
import lark

In [53]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [132]:
# This information is going to pass inside language model, so make it as descriptive as possible
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The information chunk is from, should be one of `/content/doc1.pdf`, `/content/doc2.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the doc",
        type="integer",
    ),
]

In [133]:
document_content_description = "Information about India"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [134]:
question = "Did they say anything about Tourism?"

**You will receive a warning** about predict_and_parse being deprecated the first time you executing the next line. This can be safely ignored.

In [135]:
docs = retriever.get_relevant_documents(question)

query='Tourism' filter=None limit=None


In [136]:
for d in docs:
    print(d.metadata)

{'page': 1, 'source': '/content/doc2.pdf'}
{'page': 0, 'source': '/content/doc2.pdf'}
{'page': 0, 'source': '/content/doc2.pdf'}
{'page': 0, 'source': '/content/doc2.pdf'}


In [137]:
docs[0]

Document(page_content="India's diplomatic influence is also growing on the global stage. The country actively \nparticipates in international forums and has strong bilateral relations with nations around the \nworld. India is a founding member of the Non-Aligned Movement and plays an active role in \nvarious international organizations, such as the United Nations and World Trade Organization.\nIn conclusion, India is a vast and diverse country with a rich cultural heritage, stunning \nlandscapes, and a rapidly growing economy. It is a nation where ancient traditions coexist with \nmodern aspirations. Despite its challenges, India continues to evolve and leave an indelible \nmark on the world, making it a fascinating and dynamic country to explore.", metadata={'page': 1, 'source': '/content/doc2.pdf'})

In [138]:
docs[1]

Document(page_content='However, India also faces various socio-economic challenges. Poverty, income inequality, and \nunemployment are persistent issues that the country strives to address. Efforts are being made\nto improve education, healthcare, infrastructure, and social welfare programs to uplift \nmarginalized sections of society.\nEducation plays a vital role in India, with a strong emphasis on academic excellence. The \ncountry has a vast network of schools, colleges, and universities, producing a large number of \ngraduates every year. Indian professionals have made significant contributions in various fields \nglobally, particularly in science, technology, engineering, and mathematics (STEM).\nThe Indian film industry, popularly known as Bollywood, is a global phenomenon, producing the\nlargest number of films annually. Indian cinema reflects the diversity and cultural richness of \nthe country and has a massive following both within India and among the Indian diaspora \nworld

In [139]:
docs[2]

Document(page_content="worldwide.\nIndian cuisine is renowned for its flavors, spices, and regional specialties. Each state has its \nown culinary traditions, offering a wide range of vegetarian and non-vegetarian dishes. Indian \nfood has gained international popularity, with dishes like curry, biryani, dosa, and tandoori \nbeing enjoyed by people worldwide.\nThe Indian rupee is the official currency in the Republic of India. The rupee is subdivided into \n100 paise. The issuance of the currency is controlled by the Reserve Bank of India.\n₹ The Indian rupee sign ( ) is the currency symbol for the Indian rupee the official currency of \nIndia\nTourism is a significant contributor to India's economy. The country attracts millions of visitors \neach year who come to explore its historical sites, architectural wonders, wildlife sanctuaries, \nand scenic landscapes. Iconic landmarks such as the Taj Mahal, Jaipur's palaces, Kerala's \nbackwaters, and the beaches of Goa are popular tourist 

### **Additional tricks: Compression**

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.

In [140]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [141]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [142]:
# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)

In [143]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever() # try with or without  -> search_type="mmr"
)

In [144]:
question = "What does it say about Mahatma Gandhi?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

Led by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement gained momentum and eventually led to India's independence on August 15, 1947.
----------------------------------------------------------------------------------------------------
Document 2:

Led by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement gained momentum and eventually led to India's independence on August 15, 1947.
----------------------------------------------------------------------------------------------------
Document 3:

India is a founding member of the Non-Aligned Movement and plays an active role in various international organizations, such as the United Nations and World Trade Organization.
----------------------------------------------------------------------------------------------------
Document 4:

"Efforts are being made to improve education, healthcare, infrastructure, and social welfare programs to uplift marginalized sections of s

#### Combining mmr techniques

In [145]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [146]:
question = "What does it say about Mahatma Gandhi?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

Led by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement gained momentum and eventually led to India's independence on August 15, 1947.
----------------------------------------------------------------------------------------------------
Document 2:

India is a founding member of the Non-Aligned Movement and plays an active role in various international organizations, such as the United Nations and World Trade Organization.
----------------------------------------------------------------------------------------------------
Document 3:

"Efforts are being made to improve education, healthcare, infrastructure, and social welfare programs to uplift marginalized sections of society." "Education plays a vital role in India, with a strong emphasis on academic excellence." "The country has a vast network of schools, colleges, and universities, producing a large number of graduates every year." "Indian professionals have made significant contributions i

### **Other types of retrieval**

It's worth noting that vectordb are not the only kind of tool to retrieve documents.

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as TF-IDF or SVM.

In [147]:
from langchain.retrievers import SVMRetriever
from langchain.retrievers import TFIDFRetriever
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [149]:
# Load PDF
loader = PyPDFLoader("/content/doc1.pdf")
pages = loader.load()
all_page_text=[p.page_content for p in pages]
joined_page_text=" ".join(all_page_text)

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,chunk_overlap = 150)
splits = text_splitter.split_text(joined_page_text)

In [150]:
# Retrieve
svm_retriever = SVMRetriever.from_texts(splits,embedding)
tfidf_retriever = TFIDFRetriever.from_texts(splits)

In [151]:
question = "What does it say about Mahatma Gandhi?"
docs_svm=svm_retriever.get_relevant_documents(question)
print(docs_svm[0].page_content)

people from different communities together.
The history of India is characterized by ancient civilizations, invasions, and the establishment of
powerful empires. The Indus Valley Civilization, one of the world's oldest urban civilizations, 
flourished in the northwestern part of the Indian subcontinent around 2500 BCE. Over the 
centuries, India witnessed the rise and fall of several dynasties, including the Maurya, Gupta, 
and Mughal empires. The Mughal period, in particular, left a lasting impact on Indian culture, 
art, and architecture.
India's struggle for independence from British colonial rule is a significant chapter in its history. 
Led by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement 
gained momentum and eventually led to India's independence on August 15, 1947. This day is 
celebrated annually as Independence Day.
India's economy is one of the fastest-growing in the world. It has transitioned from an agrarian 
economy to a service-oriented an

In [153]:
question = "What does it say about Mahatma Gandhi?"
docs_tfidf=tfidf_retriever.get_relevant_documents(question)
print(docs_tfidf[0].page_content)

people from different communities together.
The history of India is characterized by ancient civilizations, invasions, and the establishment of
powerful empires. The Indus Valley Civilization, one of the world's oldest urban civilizations, 
flourished in the northwestern part of the Indian subcontinent around 2500 BCE. Over the 
centuries, India witnessed the rise and fall of several dynasties, including the Maurya, Gupta, 
and Mughal empires. The Mughal period, in particular, left a lasting impact on Indian culture, 
art, and architecture.
India's struggle for independence from British colonial rule is a significant chapter in its history. 
Led by Mahatma Gandhi and other freedom fighters, the non-violent resistance movement 
gained momentum and eventually led to India's independence on August 15, 1947. This day is 
celebrated annually as Independence Day.
India's economy is one of the fastest-growing in the world. It has transitioned from an agrarian 
economy to a service-oriented an

### **Retrieval + Question Answering :  Connecting with LLMs**

In [160]:
llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [161]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [162]:
print(vectordb._collection.count())

10


In [163]:
question = "What does it say about Mahatma Gandhi?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [164]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

### **RetrievalQA chain**

In [165]:
from langchain.chains import RetrievalQA

In [166]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever())

In [167]:
result = qa_chain({"query": question})

In [168]:
result["result"]

"The context mentions that Mahatma Gandhi was a leader of the non-violent resistance movement during India's struggle for independence from British colonial rule. He played a significant role in gaining momentum for the movement, which eventually led to India's independence on August 15, 1947. Mahatma Gandhi is widely recognized as a key figure in India's history and is known for his principles of non-violence and civil disobedience."

### Prompt

In [169]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [170]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [172]:
question = "what is there about cultural heritage? Elaborate."

In [173]:
result = qa_chain({"query": question})

In [175]:
result["result"]

"India's cultural heritage is preserved in its ancient monuments and UNESCO World Heritage Sites, showcasing its rich history and artistic traditions. The country's diverse religions and languages contribute to its multicultural fabric, with festivals celebrated by different communities. India's cultural heritage is a testament to its ancient civilizations, powerful empires, and the lasting impact of the Mughal period. Thanks for asking!"

In [174]:
result["source_documents"][0]

Document(page_content="India's cultural heritage is preserved in its ancient monuments and UNESCO World Heritage \nSites. From the intricate carvings of Khajuraho temples to the majestic forts of Rajasthan, these\narchitectural marvels reflect India's rich history and artistic traditions.\nIndia's diversity extends to its languages as well. While Hindi and English are the official \nlanguages at the national level, there are 22 officially recognized regional languages, including \nBengali, Tamil, Telugu, Marathi, Urdu, Punjabi, and Gujarati, among others. This linguistic \ndiversity is a testament to India's multicultural ethos.\nIn recent years, India has made significant strides in space exploration. The Indian Space \nResearch Organization (ISRO) has successfully launched satellites and missions, including the\nMars Orbiter Mission (MOM), also known as Mangalyaan. These achievements have placed \nIndia among the elite group of nations with advanced space programs.", metadata={'page'

### RetrievalQA chain types : Map reduce, Refine, Map rerank

In [176]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [177]:
result = qa_chain_mr({"query": question})

In [178]:
result["result"]

"India's cultural heritage is a rich and diverse tapestry of traditions, art forms, architecture, religions, and cuisines that have evolved over thousands of years. It is deeply rooted in the country's history and serves as a testament to its cultural diversity and artistic traditions.\n\nOne aspect of India's cultural heritage is its ancient monuments and UNESCO World Heritage Sites. These architectural marvels, such as the intricate carvings of Khajuraho temples and the majestic forts of Rajasthan, reflect India's rich history and showcase the skills and craftsmanship of past civilizations. They not only hold historical significance but also contribute to the artistic and architectural heritage of the world.\n\nReligion also plays a significant role in India's cultural heritage. The country is the birthplace of major religions like Hinduism, Buddhism, Jainism, and Sikhism. Temples, mosques, churches, and gurudwaras can be found in every corner of the country, each reflecting unique a

### **Make it like chatbot & Adding memory**

In initial steps are similar as above : Upload your vectro db, credentials, initialize vectordb, embeddings etc

In [179]:
llm_name = "gpt-3.5-turbo"

In [180]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [181]:
question = "What are places to visit for tourists?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [186]:
docs[0].page_content

"India's cultural heritage is preserved in its ancient monuments and UNESCO World Heritage \nSites. From the intricate carvings of Khajuraho temples to the majestic forts of Rajasthan, these\narchitectural marvels reflect India's rich history and artistic traditions.\nIndia's diversity extends to its languages as well. While Hindi and English are the official \nlanguages at the national level, there are 22 officially recognized regional languages, including \nBengali, Tamil, Telugu, Marathi, Urdu, Punjabi, and Gujarati, among others. This linguistic \ndiversity is a testament to India's multicultural ethos.\nIn recent years, India has made significant strides in space exploration. The Indian Space \nResearch Organization (ISRO) has successfully launched satellites and missions, including the\nMars Orbiter Mission (MOM), also known as Mangalyaan. These achievements have placed \nIndia among the elite group of nations with advanced space programs."

In [187]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Hello world!")

'Hello! How can I assist you today?'

In [188]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

# Run chain
from langchain.chains import RetrievalQA
question = "What are places to visit for tourists?"
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

"Some popular tourist destinations in India include the Taj Mahal, Jaipur's palaces, Kerala's backwaters, and the beaches of Goa. Thanks for asking!"

### **Adding Memory**

In [190]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [191]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [192]:
question = "tell me something about India"
result = qa({"question": question})

In [193]:
result['answer']

"India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.\n\nIndia is the seventh-largest country by land area and the second-most populous country in the world, with a population exceeding 1.3 billion people. It is a federal parliamentary democratic republic, with a president as the head of state and a prime minister as the head of government. The country follows a multi-tiered administrative structure, with 28 states and 9 union territories, each having its own elected government.\n\nIndia has a rich cultural heritage that has evolved over thousands of years. It

In [194]:
question = "please list the religions followed there?"
result = qa({"question": question})

In [195]:
result['answer']

"In India, various religions are followed, including Hinduism, Islam, Christianity, Sikhism, Buddhism, and Jainism, among others. These religions coexist harmoniously, contributing to India's multicultural fabric."

In [201]:
question = "I am Ramendra, I want to visit few places there, please suggest."
result = qa({"question": question})

In [202]:
print(result['answer'])

There are numerous incredible places to visit in India. Here are some popular suggestions:

1. Taj Mahal, Agra: One of the Seven Wonders of the World, this iconic white marble mausoleum is a must-visit for its stunning architecture and romantic history.

2. Jaipur, Rajasthan: Known as the "Pink City," Jaipur offers a glimpse into the royal heritage of Rajasthan with its magnificent palaces, forts, and vibrant markets.

3. Varanasi, Uttar Pradesh: Considered one of the oldest inhabited cities in the world, Varanasi is a sacred place for Hindus. It is famous for its ghats (steps leading to the river), where people perform religious rituals and witness mesmerizing evening aarti ceremonies.

4. Kerala Backwaters: Experience the serene beauty of Kerala's backwaters by taking a houseboat cruise through the interconnected network of canals, lakes, and lagoons.

5. Goa: Known for its beautiful beaches, vibrant nightlife, and Portuguese influence, Goa is a popular destination for relaxation and

In [203]:
question = "Do you know What is my name?"
result = qa({"question": question})
result['answer']

'I am an AI language model and do not have a personal name. You can refer to me as OpenAI or ChatGPT. How can I assist you today?'